In [2]:
from pygame_2d import*
import numpy as np
import const

screen = np.zeros((720, 1280, 3), dtype = np.uint8)



pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:

high, low = 1,0

alpha = 0, 2*math.pi    # góc quay 
rVelo = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO  # vận tốc quay 
fVelo = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO    # vận tốc tiến



In [11]:
lowerBoundLidar = np.full((PLAYER_SETTING.CASTED_RAYS,), 0)
upperBoundLidar = np.full((PLAYER_SETTING.CASTED_RAYS,), INT_INFINITY)

lowState = np.array([alpha[0], fVelo[0], rVelo[0], lowerBoundLidar], dtype= object)
upState = np.array([alpha[1], fVelo[1], rVelo[1], upperBoundLidar], dtype= object)



In [13]:

gamma = 0.99
alpha = 0.1
epsilon = 1
epsilon_decay = epsilon / 4000
n_epsilondes = 10000


# set các khoảng rời rạc
ALPHA_SPACE = 3
FVELO_SPACE = 4
RVELO_SPACE = 4
BOUNDlIDAR_SPACE = 3


In [40]:

new_observation_shape = (ALPHA_SPACE, FVELO_SPACE, RVELO_SPACE, BOUNDlIDAR_SPACE)

bins = []
for i in range(3):
    item = np.linspace(
        lowState[i],
        upState[i],
        num=new_observation_shape[i],
        endpoint=False)
    item = np.delete(item, 0)
    bins.append(item)
    print(bins[i])

def get_discrete_state(s):
    new_s = []
    for i in range(3):
        new_s.append(np.digitize(s[i], bins[i]))
    return new_s

boundLidar_bin_index = np.array([0, 45, 135, 180])
boundLidar_discrete = (np.digitize([np.arange(0,360)], boundLidar_bin_index))
boundLidar_discrete


[2.0943951 4.1887902]
[-50.   0.  50.]
[-0.5  0.   0.5]


array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [16]:
q_table = np.zeros(new_observation_shape + (ACTION_SPACE, ))
print(q_table.shape)

(3, 4, 4, 3, 6)


In [ ]:

def pick_sample(state):
    if np.random.random() > epsilon:
        action = np.argmax(q_table[tuple(state)])
    else:
        action = np.random.randint(0, ACTION_SPACE)
    return action

def step(action):
    pass

reward_records = []
for i in range(n_epsilondes):
    done = False
    total_reward = 0
    
    game = PyGame2D(screen=screen)
    state = game.observe()
    
    s_dis = get_discrete_state(state)
    while not done:
        action = pick_sample(s_dis, i)
        #! ... (thêm) sau khi thực hiện action thì cập nhật lại...
        #! sửa lại hàm action bên PyGame2D 
        state, reward, done = game.observe(), game.evaluate(), game.is_done()
        
        s_dis_next = get_discrete_state(state)

        # Update Q-Table
        maxQ = np.max(q_table[tuple(s_dis_next)])
        q_table[tuple(s_dis)][action] += alpha * (reward + gamma * maxQ - q_table[tuple(s_dis)][action])

        s_dis = s_dis_next
        total_reward += reward

    # Update epsilon for each episode
    if epsilon - epsilon_decay >= 0:
        epsilon -= epsilon_decay

    # Record total rewards in episode 
    print("Run episode {} with rewards {}".format(i, total_reward), end="\r")
    reward_records.append(total_reward)

print("\nDone")
